In [21]:
#Using the CambridgeCore MacLehose pdfminer extracted text files (with hand-corrected dates), creates a CSV file tracking the metadata from the filenames (with an eye to adding more fields in the future, e.g., for genre of voyage / letter / testimony / etc)

# set working directory
import os
os.chdir('text-data/CambridgeCore MacLehose pdfminer extract')
import csv
#01_06_NNE-_0624_The_voyage_of_Bertus_into_Ireland_Anno_684_pp.010-010.pdf filename format for reference
# open the file in the write mode
with open('ledger.csv', 'w', newline = '') as ledger:
    
    # create the csv writer
    writer = csv.writer(ledger)
    
    # create header
    writer.writerow(['vol', 'chap', 'geog', 'date', 'title', 'pages'])

    filelist = os.scandir(os.getcwd())
    for entry in filelist:
        row = [entry.name[:2], entry.name[3:5], entry.name[6:10], entry.name[11:15], entry.name[16:-15], entry.name[-11:-4]]
        #print(entry.name, '\n', row)
        # write a row to the csv file
        writer.writerow(row)

In [21]:
# creates cat_mod corpus with expl, enc, and nat fields encoded into filename
# code below still misses a couple of files--but better than the initial version, which I apparently wrote to just catch enc/expl entries, and totally forgot about that filter
old_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat'
new_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_mod'

import csv
import os
import re


with open('text-data/ledger_merged_2.csv', mode = 'r') as ledger_csv:
    ledger = csv.DictReader(ledger_csv)
    for row in ledger:
        if row['geog'] == 'AMER':
            filelist = os.scandir(old_path)
            for entry in filelist:
                #print('scanning' , entry.name)
                #print ( 'condition checks: ', row['vol'].zfill(2), re.search(r'_(.*?)_', entry.name).group(1), row['vol'].zfill(2) == entry.name[:2], row['chap'] == re.search(r'_(.*?)_', entry.name).group(1))
                if row['vol'].zfill(2) == entry.name[:2] and (row['chap'].zfill(2) == re.search(r'_(.*?)_', entry.name).group(1) or row['chap'].zfill(2) == re.search(r'_(.*?)_', entry.name).group(1)):
                    #print(entry.name, 'passed')
                    # find the relevant text file
                    with open(entry.path, 'r', encoding="utf8") as f:
                        text = f.read()
                        expl = 'expl_y' if row['expl'] == 'v' else 'expl_n'
                        if row['enc'] == 'v' and row['expl'] != 'v':
                            enc = 'enc_full'
                        elif row['enc'] == 'v':
                            enc = 'enc_part'
                        elif row['enc_min'] == 'v':
                            enc = 'enc_min'
                        else:
                            enc = 'enc_none'
                        # determine name factors
                        name = row['vol'].zfill(2) + '_' + row['chap'].zfill(2) + '-' + row['date'] + '-' + row['nat'] + '-' + enc + '-' + expl + '-' + row['title'] + '-' + row['pages'].replace('-', '_') + '.txt'
                        # string formatted filename
                        with open((new_path + '/' + name), 'w', encoding="utf8") as fw:
                            fw.write(text)
                            #print('text outputted')
                        #break